In [1]:
import json
import argparse
import os
import re
from glob import glob
import gzip
import pandas as pd
import numpy as np
from bids import BIDSLayout, BIDSLayoutIndexer


In [ ]:
bidsDir='..'
idx = BIDSLayoutIndexer(ignore='*.json')
bidsLayout = BIDSLayout(bidsDir, indexer=idx)

In [3]:
# aggregate fmap info
def fmap_csv(bidsLayout):
    df = pd.DataFrame(columns=['filename','type','description','phase_dir','phase_ge','acqtime'])#,'tr','slice','mb'])
    for sub in bidsLayout.get_subjects():
        for jBids in bidsLayout.get(subject=sub,extension='.json',suffix=['bold','epi']):
            j = json.loads(jBids.get_json())
            jFile = jBids.path
            df.loc[len(df.index)] = [jBids.filename, jBids.entities['datatype'], j['SeriesDescription'],
                                     j['PhaseEncodingDirection'], j['PhaseEncodingPolarityGE'],
                                     j['AcquisitionTime']]#,j['RepetitionTime'],j['SliceThickness'],j['MultibandAccelerationFactor']]
    df.to_csv('ambly_fmap_info.csv',index=False)

In [17]:
def fix_fmap1(bidsLayout):
    for file in bidsLayout.get(suffix='epi',extension='.json'):
        if file.entities['direction'] == 'PA':
            # reconstruct 
            boldName = os.path.basename(file.path).replace('.json','.nii.gz').replace('_acq','_task').replace('_epi','_bold').replace('_dir-PA','')
            boldPath = os.path.relpath(file.path, bidsDir)
            boldPath = os.path.join('func', boldName)
            if not os.path.isfile(os.path.join(bidsDir, f'sub-{file.entities["subject"]}', boldPath)):
                print(f'ERROR:\t No {boldPath}')
            else:
                j = json.loads(file.get_json())
                j['IntendedFor'] = boldPath
                with open(file.path, 'w') as fid:
                    json.dump(j,fid, indent=4)

In [37]:
#def fix_fmap_toptup(bidsLayout):
sub_all = bidsLayout.get_subjects()
for sub in sub_all:
    # get bold
    bold = [b.relpath.replace(f'sub-{sub}/','') for b in bidsLayout.get(subject=sub, suffix='bold', extension='.nii.gz')]
    # add bold intended for to each topup json
    for bidsEpi in bidsLayout.get(subject=sub, suffix='epi', extension='json'):
        j = json.loads(bidsEpi.get_json())
        if 'topup' in j['SeriesDescription'].lower():
            j['IntendedFor'] = bold
            with open(bidsEpi.path, 'w') as fid:
                json.dump(j, fid, indent=4)


'/scratch/st-hyim1-1/ambly/code/../sub-31/fmap/sub-31_dir-AP_epi.json'

In [8]:
sub='04'

In [11]:
func = bidsLayout.get(suffix='bold',extension='.dtseries.nii')


[<BIDSImageFile filename='/scratch/st-hyim1-1/ambly/code/../derivatives/sub-05/func/sub-05_task-locMT_space-fsLR_den-91k_bold.dtseries.nii'>,
 <BIDSImageFile filename='/scratch/st-hyim1-1/ambly/code/../derivatives/sub-05/func/sub-05_task-locObj_space-fsLR_den-91k_bold.dtseries.nii'>,
 <BIDSImageFile filename='/scratch/st-hyim1-1/ambly/code/../derivatives/sub-05/func/sub-05_task-motion1_space-fsLR_den-91k_bold.dtseries.nii'>,
 <BIDSImageFile filename='/scratch/st-hyim1-1/ambly/code/../derivatives/sub-05/func/sub-05_task-motion2_space-fsLR_den-91k_bold.dtseries.nii'>,
 <BIDSImageFile filename='/scratch/st-hyim1-1/ambly/code/../derivatives/sub-05/func/sub-05_task-retino_space-fsLR_den-91k_bold.dtseries.nii'>,
 <BIDSImageFile filename='/scratch/st-hyim1-1/ambly/code/../derivatives/sub-06/func/sub-06_task-locMT_space-fsLR_den-91k_bold.dtseries.nii'>,
 <BIDSImageFile filename='/scratch/st-hyim1-1/ambly/code/../derivatives/sub-06/func/sub-06_task-locObj_space-fsLR_den-91k_bold.dtseries.nii'>,